In [26]:
# Importar bibliotecas e  helpers

from helpers.helpers import (Data, lista_diretorio_range)
import pandas as pd
import fastparquet 
import datetime 


In [27]:
# Definindo range de data através da função "Data"

Data_ini = Data(25)
Data_fim = Data(25)


print(Data_ini,Data_fim)

2023-09-30 2023-09-30


In [28]:
# Lendo os arquivos que preciso do diretório especificado no range selecionado

diretorio_arquivo = lista_diretorio_range("F:/Documentos/07.PYTHON/PROJETOS_MEF/INADIMPLENCIA/HH/", 'Bases_Acionamentos', '.parquet', Data_ini, Data_fim)

diretorio_arquivo

['F:/Documentos/07.PYTHON/PROJETOS_MEF/INADIMPLENCIA/HH/Bases_Acionamentos2023-09-30.parquet']

In [29]:
# Lendo arquivo com fastparquet e conertendo para pandas

Pf = fastparquet.ParquetFile(diretorio_arquivo)

Hora_Hora = Pf.to_pandas()



In [30]:
# Filtros inciais 

Filtro_hora_hora = Hora_Hora[(Hora_Hora['Atendidas'] >= 1) & (Hora_Hora['Perfil'] == 'Digital')]


In [31]:
# Inserir coluna mês no df
# Garantir que o campo esteja como data

meses_em_portugues = {
    'January': 'Janeiro',
    'February': 'Fevereiro',
    'March': 'Março',
    'April': 'Abril',
    'May': 'Maio',
    'June': 'Junho',
    'July': 'Julho',
    'August': 'Agosto',
    'September': 'Setembro',
    'October': 'Outubro',
    'November': 'Novembro',
    'December': 'Dezembro'
}


Filtro_hora_hora['Data Acionamento'] = pd.to_datetime(Filtro_hora_hora['Data Acionamento'])

# Adicionando a coluna
Filtro_hora_hora['Mes'] = Filtro_hora_hora['Data Acionamento'].dt.strftime('%B').map(meses_em_portugues)



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [32]:
# Fazer Tratativa no campo de filtro ativação 

Valor_x = 'BASE'

Filtro_hora_hora.loc[Filtro_hora_hora['Filtro Ativação'].isnull(), 'Filtro Ativação'] = Valor_x

Filtro_hora_hora.head()


,cust code,Inicio da ligação,Data Hora Fim,Usuario de inclusão,Tabulação,Telefone Acionado,Valor cliente,Valor acordo/Promessa,Produto,Segmento,...,OPERAÇÃO,Alega pagamento,Fornecedor,R$ Atendidas,R$ CPC,R$ Alega,R$ Sucesso,R$ CPCA,CAMPO CHAVE,Mes
290,1.53731253,2023-09-30 12:40:19,2023-09-30 12:40:19,TIM.FRANCISCA.3891,Pagamento no Sistema,11985588800,93.73,None,Consumer,Pós Puro,...,A/R,0.0,Vcom,93.73,93.73,0.0,0.0,93.73,1.537312532023-09-30 12:40:192023-09-30 12:40:...,Setembro
310,1.261960751,2023-09-30 13:25:03,2023-09-30 13:25:03,TIM.DOUGLAS.3914,Pagamento no Sistema,11985313023,64.99,None,Consumer,Controle - 31 a 60 - Digital,...,A/R,0.0,Vcom,64.99,64.99,0.0,0.0,64.99,1.2619607512023-09-30 13:25:032023-09-30 13:25...,Setembro
334,1.317778580,2023-09-30 11:42:12,2023-09-30 11:42:12,TIM.ADRIANNY.4129,Reclamação de conta,11973170139,60.23,None,Consumer,Controle,...,A/R,0.0,Vcom,60.23,60.23,0.0,0.0,60.23,1.3177785802023-09-30 11:42:122023-09-30 11:42...,Setembro
337,1.87652621,2023-09-30 20:37:55,2023-09-30 20:37:55,TIM.GABRIEL.4228,Reclamação de conta,48984223431,71.99,None,Consumer,Controle,...,A/R,0.0,Vcom,71.99,71.99,0.0,0.0,71.99,1.876526212023-09-30 20:37:552023-09-30 20:37:...,Setembro
342,1.76604961,2023-09-30 14:24:54,2023-09-30 14:24:54,TIM.GABRIEL.4228,Reclamação de conta,11952157515,89.16,None,Consumer,Controle,...,A/R,0.0,Vcom,89.16,89.16,0.0,0.0,89.16,1.766049612023-09-30 14:24:542023-09-30 14:24:...,Setembro


In [33]:
# Agrupamento de dados e contagem de colunas 

Dados_agrupados = Filtro_hora_hora.groupby(['Data Acionamento', 'Mes', 'cust code', 'Tabulação', 'Produto', 'Filtro Ativação'])['HORA'].count()

# Tranformar em dataframe resetando o index

Dados_agrupados = Dados_agrupados.reset_index()

# Renomenado HORA para coluna de contagem

Dados_agrupados = Dados_agrupados.rename(columns={'HORA': 'Quantidade'})


Dados_agrupados



,Data Acionamento,Mes,cust code,Tabulação,Produto,Filtro Ativação,Quantidade
0,2023-09-30,Setembro,1.10300868,SEM INTERAÇÃO,Consumer,BASE,1
1,2023-09-30,Setembro,1.10371187,SEM INTERAÇÃO,Consumer,BASE,1
2,2023-09-30,Setembro,1.10415505,E O CLIENTE,Consumer,BASE,1
3,2023-09-30,Setembro,1.10505663,Desligou sem interacao,Consumer,BASE,1
4,2023-09-30,Setembro,1.10593291,PROMESSA A VISTA,Consumer,BASE,1
...,...,...,...,...,...,...,...
144341,2023-09-30,Setembro,1.88758775,E O CLIENTE,Consumer,BASE,1
144342,2023-09-30,Setembro,1.88759229,SEM INTERAÇÃO,Consumer,BASE,1
144343,2023-09-30,Setembro,7.1923221.10,SEM INTERAÇÃO,Consumer,BASE,2
144344,2023-09-30,Setembro,7.1939091.10,E O CLIENTE,Consumer,BASE,1


In [34]:
# Realizando cruzamento de dados com o de para de finalizações 

# Lendo de para
De_para = pd.read_excel('F:/Documentos/07.PYTHON/PROJETOS_MEF/DE_PARA.xlsx')

# Cruzamento
Cruzamento_dados = pd.merge(Dados_agrupados, De_para, left_on= 'Tabulação', right_on= 'Field1', how= 'inner')

# Deixando somente colunas do meu Df principal (Dados_agrupados)

colunas_ok = Dados_agrupados.columns

Cruzamento_dados = Cruzamento_dados[colunas_ok]

Cruzamento_dados



,Data Acionamento,Mes,cust code,Tabulação,Produto,Filtro Ativação,Quantidade
0,2023-09-30,Setembro,1.11285933,ALEGA PAGAMENTO,Consumer,BASE,1
1,2023-09-30,Setembro,1.13555500,ALEGA PAGAMENTO,Consumer,BASE,1
2,2023-09-30,Setembro,1.14210068,ALEGA PAGAMENTO,Consumer,BASE,1
3,2023-09-30,Setembro,1.17355073,ALEGA PAGAMENTO,Consumer,BASE,1
4,2023-09-30,Setembro,1.17575278,ALEGA PAGAMENTO,Consumer,BASE,1
...,...,...,...,...,...,...,...
4030,2023-09-30,Setembro,1.328133935,Recusa nao informado - Sem interacao,Consumer,BASE,1
4031,2023-09-30,Setembro,1.50650197,Recusa nao informado - Sem interacao,Consumer,BASE,1
4032,2023-09-30,Setembro,1.74483625,Recusa nao informado - Sem interacao,Consumer,BASE,1
4033,2023-09-30,Setembro,1.87200913,Recusa nao informado - Sem interacao,Consumer,BASE,1


In [35]:
# Inserindo Colunas adicionais sem informações

Novas_colunas = ['Credor', 'Cpf', 'Cod', 'Situação']

for colunas in Novas_colunas:
    Cruzamento_dados[colunas] = [None] * len(Cruzamento_dados)


Cruzamento_dados

,Data Acionamento,Mes,cust code,Tabulação,Produto,Filtro Ativação,Quantidade,Credor,Cpf,Cod,Situação
0,2023-09-30,Setembro,1.11285933,ALEGA PAGAMENTO,Consumer,BASE,1,None,None,None,None
1,2023-09-30,Setembro,1.13555500,ALEGA PAGAMENTO,Consumer,BASE,1,None,None,None,None
2,2023-09-30,Setembro,1.14210068,ALEGA PAGAMENTO,Consumer,BASE,1,None,None,None,None
3,2023-09-30,Setembro,1.17355073,ALEGA PAGAMENTO,Consumer,BASE,1,None,None,None,None
4,2023-09-30,Setembro,1.17575278,ALEGA PAGAMENTO,Consumer,BASE,1,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
4030,2023-09-30,Setembro,1.328133935,Recusa nao informado - Sem interacao,Consumer,BASE,1,None,None,None,None
4031,2023-09-30,Setembro,1.50650197,Recusa nao informado - Sem interacao,Consumer,BASE,1,None,None,None,None
4032,2023-09-30,Setembro,1.74483625,Recusa nao informado - Sem interacao,Consumer,BASE,1,None,None,None,None
4033,2023-09-30,Setembro,1.87200913,Recusa nao informado - Sem interacao,Consumer,BASE,1,None,None,None,None


In [36]:
# Saida final

# Definindo diretorio + nome e extensão do arquivo

nome = 'F:/Documentos/07.PYTHON/PROJETOS_MEF/INADIMPLENCIA/OUTPUT/INADIMPLENCIA_DIGITAL_'
ext = '.xlsx'
data_ok = pd.to_datetime(Data_ini)
mes = data_ok.strftime('%B')

# Salvando o arquivo + mes

Cruzamento_dados.to_excel(f'{nome}-{mes}{ext}', index = False)
